In [1]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import os
import cv2


2024-07-31 12:18:40.842948: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-31 12:18:40.879520: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-31 12:18:40.879551: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-31 12:18:40.880237: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-31 12:18:40.885109: I tensorflow/core/platform/cpu_feature_guar

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications.vgg16 import VGG16
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

In [3]:
# GPU settings



gpus = tf.config.list_physical_devices('GPU')



# Setting gpu[0] as main device
if gpus:
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)


# Setting memory growth
try:
  tf.config.experimental.set_memory_growth(gpus[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass



# tf.debugging.set_log_device_placement(True)


os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
print(os.getenv('TF_GPU_ALLOCATOR'))

1 Physical GPUs, 1 Logical GPU
cuda_malloc_async


2024-07-31 12:18:43.062899: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-31 12:18:43.090404: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-31 12:18:43.090563: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
#Initialization of selective search
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

In [5]:
#Path to images and folder with bounding boxes of all airplane locations in an image for all images
images_path = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Aiplanes_dataset_for_rcnn/Images"
label_path = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Aiplanes_dataset_for_rcnn/Airplanes_Annotations"

In [6]:
#there are 700 images with name starting with airplane, and 33 with numbers
#to hold the image name and their bb of all airplane locations
img_bb = {}

In [7]:
for label in os.listdir(label_path):                        # It gives each element of label_path directory to here (label)
    if label.split("_")[0] == "airplane":                   # We are only considering these images
        try:
            file = pd.read_csv(label_path+"/"+label)        # Reading each csv file containing bb of each image
            # print("label: ", label)
            # print("len of file: ", len(file))
            labels_for_image = []                           # To store the bb in integer format from csv
            for row in file.iterrows():
                # for k in row:
                #     print(row[k][0].split(" "))
                bb_each = list(row[1])[0].split(" ")
                for j in range(len(bb_each)):
                    bb_each[j] = eval(bb_each[j])

                labels_for_image.append(bb_each)            # Labels_for_image is the list with all bb of current image
            img_bb[label.split(".")[0]] = labels_for_image  # We choose file name without jpg as key and labels_for_image as their value
        except Exception as e:
            print(e)
            print("Error in :", label)

In [8]:
# img_bb is a dictionary that now contains image name as key and their true bb of each airplane as a list of lists as the value of dictionary
print("Total images for training: ", len(img_bb))

Total images for training:  700


In [9]:
# To check how data is stored
c = 0
for i in img_bb:
    if c == 4:
        break
    print("Image name: ", i)
    print("Number of labels: ", len(img_bb[i]))
    print("Labels: ", img_bb[i])
    c += 1

Image name:  airplane_201
Number of labels:  26
Labels:  [[22, 78, 31, 90], [29, 69, 48, 81], [44, 65, 59, 76], [58, 57, 68, 70], [122, 16, 138, 26], [134, 9, 146, 21], [93, 35, 102, 46], [79, 41, 93, 56], [69, 48, 79, 59], [27, 110, 39, 121], [38, 102, 48, 111], [48, 97, 62, 105], [61, 89, 76, 97], [73, 82, 86, 91], [87, 75, 96, 84], [106, 69, 118, 79], [118, 53, 133, 65], [133, 47, 145, 58], [144, 39, 154, 48], [116, 238, 127, 253], [126, 232, 143, 248], [136, 225, 152, 241], [154, 216, 164, 226], [165, 209, 179, 219], [166, 220, 174, 230], [152, 229, 165, 237]]
Image name:  airplane_407
Number of labels:  3
Labels:  [[30, 90, 78, 131], [143, 6, 190, 46], [200, 180, 253, 236]]
Image name:  airplane_343
Number of labels:  2
Labels:  [[122, 153, 197, 247], [79, 5, 147, 98]]
Image name:  airplane_520
Number of labels:  4
Labels:  [[63, 116, 92, 137], [107, 91, 132, 115], [140, 128, 165, 156], [176, 230, 202, 254]]


In [10]:
ele1n = ""
elebb1 = []

In [11]:
# To calculate the area of intersection
def gt(bb1, bb2):
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [12]:
'''
1   Reading each image from folder
2   we consider only images starting with name airplane
3   Performing selective search
4   Taking each region proposal of ss, Each region propasal of ss is given as an array as [x, y, w, h]
        where (x, y) are top left corner coordinates of bb and w, h are width and height
5   Here we do not add width and height as the label data cause:
6   we downloaded has bb of the format [x1, y1, x2, y2]
        (x1, y1): top left corner coordinates
        (x2, y2): bottom right corner coordinates
7   Here iou is greater than 0.7, if we haven't added 30 of this class to train_images:
        add the resized image of this proposed bounding box by selective search to train_images



Steps involved in displaying an image in opencv:
    cv2.imshow("imout: ", imout)
    cv2.waitKey(0),
    cv2.destroyAllWindows()


'''

'\n1   Reading each image from folder\n2   we consider only images starting with name airplane\n3   Performing selective search\n4   Taking each region proposal of ss, Each region propasal of ss is given as an array as [x, y, w, h]\n        where (x, y) are top left corner coordinates of bb and w, h are width and height\n5   Here we do not add width and height as the label data cause:\n6   we downloaded has bb of the format [x1, y1, x2, y2]\n        (x1, y1): top left corner coordinates\n        (x2, y2): bottom right corner coordinates\n7   Here iou is greater than 0.7, if we haven\'t added 30 of this class to train_images:\n        add the resized image of this proposed bounding box by selective search to train_images\n\n\n\nSteps involved in displaying an image in opencv:\n    cv2.imshow("imout: ", imout)\n    cv2.waitKey(0),\n    cv2.destroyAllWindows()\n\n\n'

In [13]:
#we pass each image for selective search and calculate iou for each bb output of ss out of 2000 proposals

x_new_images, y_new_labels = [], []
no_images = 0
no_images_to_be_used_for_extraction = 30
total_images_extracted = 0
for img in os.listdir(images_path):             # 1
    if no_images == no_images_to_be_used_for_extraction:   # We are only 10 images, 30 resized images each for foreground and background are taken from each of these 10 images
                                                # Total of 300 positive images and 300 negative samples are given for VGGNet to train
        break
    try:
        if img.split("_")[0] == "airplane":                              # 2

            image = cv2.imread(os.path.join(images_path, img))
            print("image original shape: ", image.shape)
            ss.setBaseImage(image)                                       # 3
            ss.switchToSelectiveSearchFast(base_k=10)
            ssresults = ss.process()
            imout = image.copy()
            print("imout shape: ", imout.shape)
            # cv2.imshow("imout: ", imout)
            # cv2.waitKey(0),
            # cv2.destroyAllWindows()


            # ele1n = os.path.join(images_path, img)

            # In openCV, consider a square window facing us, 
            # its left side will be y axis and top side will be x axis
            # top left corner is considered as (0, 0)

            iou_07 = 0
            iou_03 = 0
            for i in ssresults:
                x1, y1, w, h = i                                          # 4
                bb2 = {"x1":x1, "y1":y1, "x2":x1+w, "y2":y1+h}            
                img = img.split(".")[0]
                for j in img_bb[img]:                                     # For each true bb in img_bb, we calculate iou for the bb region proposed by ss
                    x, y, xn, yn = j                                      # 5
                    bb1 = {"x1":x, "y1":y, "x2":xn, "y2":yn}              # 6
                    iou = gt(bb1, bb2)                                    # Calculating iou
                    if iou > 0.7 and iou_07 < 30:                         # 7
                        # print("iou07: bb: x1, y1, w, h, x1+w, y1+h:", x1, y1, w, h, x1+w, y1+h)
                        timage = imout[y1:y1+h,x1:x1+w]
                        print("timage shape: ", timage.shape)
                        resized = cv2.resize(timage, (224, 224), interpolation=cv2.INTER_AREA)
                        print("resized timage shape: ", resized.shape)
                        x_new_images.append(resized)
                        y_new_labels.append(1)
                        iou_07 += 1
                        # cv2.imshow("resized: ", resized)
                        # cv2.waitKey(0)
                        # cv2.destroyAllWindows()
                        # cv2.rectangle(image, (bb1["x1"], bb1["y1"]), (bb1["x2"], bb1["y2"]), (255, 0, 0), 2)
                        # cv2.rectangle(image, (bb2["x1"], bb2["y1"]), (bb2["x2"], bb2["y2"]), (0, 255, 0), 1)
                    elif iou < 0.3 and iou_03 < 30:
                        timage = imout[y1:y1+h, x1:x1+h]
                        resized = cv2.resize(timage, (224, 224), interpolation=cv2.INTER_AREA)
                        x_new_images.append(resized)
                        y_new_labels.append(0)
                        iou_03 += 1
            # cv2.imshow(img, image)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
            # print("for image: ", img, " there are _ iou07 and _ iou03: ", iou_07, iou_03)
            total_images_extracted += iou_03 + iou_07
        no_images += 1
    except Exception as e:
        print(e)
        print("Error in image: ", img)
        continue

image original shape:  (256, 256, 3)
imout shape:  (256, 256, 3)
timage shape:  (48, 47, 3)
resized timage shape:  (224, 224, 3)
timage shape:  (47, 47, 3)
resized timage shape:  (224, 224, 3)
timage shape:  (33, 52, 3)
resized timage shape:  (224, 224, 3)
timage shape:  (44, 39, 3)
resized timage shape:  (224, 224, 3)
timage shape:  (29, 40, 3)
resized timage shape:  (224, 224, 3)
timage shape:  (51, 39, 3)
resized timage shape:  (224, 224, 3)
timage shape:  (30, 49, 3)
resized timage shape:  (224, 224, 3)
image original shape:  (256, 256, 3)
imout shape:  (256, 256, 3)
timage shape:  (40, 55, 3)
resized timage shape:  (224, 224, 3)
timage shape:  (40, 59, 3)
resized timage shape:  (224, 224, 3)
timage shape:  (29, 30, 3)
resized timage shape:  (224, 224, 3)
timage shape:  (44, 59, 3)
resized timage shape:  (224, 224, 3)
timage shape:  (65, 54, 3)
resized timage shape:  (224, 224, 3)
timage shape:  (28, 30, 3)
resized timage shape:  (224, 224, 3)
timage shape:  (52, 53, 3)
resized tim

In [14]:
# These lines can display the image you want

# i = cv2.imread("/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Aiplanes_dataset_for_rcnn/Images/airplane_343.jpg")
# i = cv2.imread("/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Aiplanes_dataset_for_rcnn/Images/42845.jpg")

# cv2.imshow("image", i)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [15]:
# Now we have in total 1246 images of positive and negative samples mix to fine tune the next VGGNet model

print("total images extracted and resized: ", total_images_extracted)
print("len of x_new_images: ", len(x_new_images))
print("len of y_new_labels: ", len(y_new_labels))

total images extracted and resized:  1246
len of x_new_images:  1246
len of y_new_labels:  1246


In [16]:
# Displaying images of x_new_images resized images:

# for index, i in enumerate(x_new_images):
#     cv2.imshow("image: " + str(y_new_labels[index]), i)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

In [17]:
# Converting resized images to array by numpy to fine tune model

x_new = np.array(x_new_images)
y_new = np.array(y_new_labels)

In [18]:
# arr = np.array(y_new)

# lb = LabelBinarizer()
# lb.fit(range(max(arr)+1))
# encoded_y = lb.transform(arr)

In [19]:
# for i in encoded_y:
#     print(i, "\n")

In [20]:
arr = np.array(y_new)
C = tf.constant(2)

encoded_y = tf.one_hot(arr, C, axis=-1)

print(encoded_y)

tf.Tensor(
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]], shape=(1246, 2), dtype=float32)


In [21]:
print("xnew: ", len(x_new))
print("xnew shape: ", x_new.shape)
print("enc_y", len(encoded_y))

xnew:  1246
xnew shape:  (1246, 224, 224, 3)
enc_y 1246


In [22]:
%load_ext tensorboard

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import SGD, Adam
import datetime

In [24]:
#defining model

class AlexNet(Sequential):
    def __init__(self, input_shape, num_classes):
        super().__init__()

        self.add(Conv2D(
            96, 
            kernel_size=(11, 11), 
            strides=4, 
            padding="valid", 
            activation="relu", 
            input_shape=input_shape,
            kernel_initializer="he_normal"
        ))
        
        self.add(MaxPooling2D(
            pool_size=(3, 3),
            strides = (2, 2),
            padding = "valid",
            data_format = None
        ))

        self.add(Conv2D(
            256,
            kernel_size=(5, 5),
            strides=1,
            padding="same",
            activation="relu",
            kernel_initializer="he_normal"
        ))

        self.add(MaxPooling2D(
            pool_size=(3, 3),
            strides = (2, 2),
            padding="valid",
            data_format=None
        ))

        self.add(Conv2D(
            384,
            kernel_size=(3, 3),
            strides=1,
            padding="same",
            activation="relu",
            kernel_initializer="he_normal"
        ))

        self.add(Conv2D(
            384,
            kernel_size=(3, 3),
            strides=1,
            padding="same",
            activation="relu",
            kernel_initializer="he_normal"
        ))

        self.add(Conv2D(
            256,
            kernel_size=(3, 3),
            strides=1,
            padding="same",
            activation="relu",
            kernel_initializer="he_normal"
        ))

        self.add(MaxPooling2D(
            pool_size=(3, 3),
            strides=(2, 2),
            padding="valid",
            data_format=None
        ))

        self.add(Dropout(0.5))
        self.add(Flatten())
        self.add(Dropout(0.5))
        self.add(Dense(4096, activation="relu"))
        self.add(Dense(4096, activation="relu"))
        self.add(Dense(1000, activation="relu"))
        self.add(Dense(num_classes, activation="softmax"))

        self.compile(
            optimizer=Adam(0.0001),
            loss = 'categorical_crossentropy',
            metrics=["accuracy"]
        )


In [25]:
model = AlexNet((224, 224, 3), 2)
model.summary()

Model: "alex_net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 54, 54, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2  (None, 26, 26, 96)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 256)       614656    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 12, 12, 256)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 384)       885120    
                                                                 
 conv2d_3 (Conv2D)           (None, 12, 12, 384)       132

In [26]:
model_dir = "/home/abhinav/Documents/Work/Hobby/Models/myalexnet.tf"

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
x_new.shape

(1246, 224, 224, 3)

In [29]:
x_train, x_test, y_train, y_test = x_new[:1000], x_new[1000:1246], encoded_y[:1000], encoded_y[1000:1246]

In [30]:
epochs = 30
batch_size = 178
train_num = 1246


In [31]:
# run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)

In [46]:
log_dir = "./logs/fit" + datetime.datetime.now().strftime("%d%m%Y-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
callbacks_list = [tensorboard_callback]

model.fit(
    x=x_train,
    y = y_train,
    epochs=epochs,
    validation_split = 0.3,
    steps_per_epoch=train_num//batch_size,
    callbacks=callbacks_list,
    verbose=1
)

Epoch 1/30
7/7 [==============================] - 18s 399ms/step - loss: 0.0913 - accuracy: 0.9729 - val_loss: 0.1590 - val_accuracy: 0.9600
Epoch 2/30
7/7 [==============================] - 1s 167ms/step - loss: 0.1176 - accuracy: 0.9600 - val_loss: 0.0897 - val_accuracy: 0.9900
Epoch 3/30
7/7 [==============================] - 1s 166ms/step - loss: 0.0984 - accuracy: 0.9671 - val_loss: 0.1880 - val_accuracy: 0.9433
Epoch 4/30
7/7 [==============================] - 1s 167ms/step - loss: 0.1286 - accuracy: 0.9514 - val_loss: 0.1282 - val_accuracy: 0.9467
Epoch 5/30
7/7 [==============================] - 1s 165ms/step - loss: 0.1262 - accuracy: 0.9586 - val_loss: 0.0751 - val_accuracy: 0.9833
Epoch 6/30
7/7 [==============================] - 1s 166ms/step - loss: 0.1510 - accuracy: 0.9457 - val_loss: 0.2498 - val_accuracy: 0.9300
Epoch 7/30
7/7 [==============================] - 1s 163ms/step - loss: 0.1443 - accuracy: 0.9514 - val_loss: 0.1769 - val_accuracy: 0.9400
Epoch 8/30
7/7 [===

In [33]:
# vggmodel = VGG16(weights='imagenet', include_top=True)

In [34]:
# for layers in (vggmodel.layers)[:15]:
#     print(layers)
#     layers.trainable = False
#     X= vggmodel.layers[-2].output
# predictions = Dense(2, activation="softmax")(X)
# model_final = Model(vggmodel.input, predictions)
# opt = Adam(lr=0.0001)
# model_final.compile(loss = keras.losses.categorical_crossentropy, optimizer = opt, metrics=["accuracy"])
# model_final.summary()

In [35]:
# class MyLabelBinarizer(LabelBinarizer):
#     def transform(self, y):
#         Y = super().transform(y)
#         if self.y_type_ == 'binary':
#             return np.hstack((Y, 1-Y))
#         else:
#             return Y
#     def inverse_transform(self, Y, threshold=None):
#         if self.y_type_ == 'binary':
#             return super().inverse_transform(Y[:, 0], threshold)
#         else:
#             return super().inverse_transform(Y, threshold)
# lenc = MyLabelBinarizer()
# Y =  lenc.fit_transform(y_new)
# X_train, X_test , y_train, y_test = train_test_split(x_new,Y,test_size=0.10)

In [36]:
# trdata = ImageDataGenerator(
#     horizontal_flip=True, 
#     vertical_flip=True, 
#     rotation_range=90
#     )
# traindata = trdata.flow(
#     x=X_train, 
#     y=y_train
#     )
# tsdata = ImageDataGenerator(
#     horizontal_flip=True, 
#     vertical_flip=True, 
#     rotation_range=90
#     )
# testdata = tsdata.flow(
#     x=X_test, 
#     y=y_test
#     )

In [37]:
# from keras.callbacks import ModelCheckpoint, EarlyStopping
# checkpoint = ModelCheckpoint(
#     "ieeercnn_vgg16_1.h5", 
#     monitor='val_loss', 
#     verbose=1, 
#     save_best_only=True, 
#     save_weights_only=False, 
#     mode='auto', 
#     period=1)
# early = EarlyStopping(
#     monitor='val_loss', 
#     min_delta=0, 
#     patience=100, 
#     verbose=1, 
#     mode='auto'
#     )
# hist = model_final.fit(
#     traindata, 
#     steps_per_epoch= 10, 
#     epochs= 100, 
#     validation_data= testdata, 
#     validation_steps=10, 
#     callbacks=[checkpoint,early]
#     )

In [38]:
c = 0
for img in os.listdir(images_path):
    if img.startswith("4"):
        c+= 1
print("Number of images that starts with 4 : ", c)

Number of images that starts with 4 :  17


In [48]:
c = 0
for img in os.listdir(images_path):
    if img.startswith("4"):
        image = cv2.imread(os.path.join(images_path, img))
        # cv2.imshow("image", image)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        ss.setBaseImage(image)
        ss.switchToSelectiveSearchFast()
        ssresults = ss.process()
        imout = image.copy()
        for bb in ssresults:
            x, y, w, h = bb
            im_to_model = imout[y:y+h, x:x+w]
            # print("shape: ", im_to_model.shape)
            im_to_m = cv2.resize(im_to_model, (224, 224) , interpolation=cv2.INTER_AREA)
            # print("r shape: ", im_to_m.shape)
            d = np.expand_dims(im_to_m, axis=0)
            # print("shape of input to model: ", d.shape)
            res = model.predict(d)
            # print("res: ", res)
            predictions = np.argmax(res, axis=1)
            # print(predictions)
            if res[0][1] > 0.97:
                cv2.rectangle(imout, (x, y), (x+w, y+h), (0, 0, 255), 1)
                c += 1
            
            # cv2.imshow("image", im_to_model)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
        cv2.imshow(str(c), imout)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

1/1 [==============================] - 0s 14ms/step


In [51]:
# '''
# 1   Reading each image from folder
# 2   we consider only images starting with name airplane
# 3   Performing selective search
# 4   Taking each region proposal of ss, Each region propasal of ss is given as an array as [x, y, w, h]
#         where (x, y) are top left corner coordinates of bb and w, h are width and height
#         to draw rectangles in cv2 we need to give two points: top left and bottom right coordinates
# 5   Here we do not add width and height as the label data cause:
# 6   we downloaded has bb of the format [x1, y1, x2, y2]
#         (x1, y1): top left corner coordinates
#         (x2, y2): bottom right corner coordinates
# 7   Here iou is greater than 0.7, if we haven't added 30 of this class to train_images:
#         add the resized image of this proposed bounding box by selective search to train_images



# Steps involved in displaying an image in opencv:
#     cv2.imshow("imout: ", imout)
#     cv2.waitKey(0),
#     cv2.destroyAllWindows()


# '''

In [49]:
# #we pass each image for selective search and calculate iou for each bb output of ss out of 2000 proposals

# x_new_images, y_new_labels = [], []
# no_images = 0
# no_images_to_be_used_for_extraction = 30
# total_images_extracted = 0
# for img in os.listdir(images_path):             # 1
#     if no_images == no_images_to_be_used_for_extraction:   # We are only 10 images, 30 resized images each for foreground and background are taken from each of these 10 images
#                                                 # Total of 300 positive images and 300 negative samples are given for VGGNet to train
#         break
#     try:
#         if img.split("_")[0] == "airplane":                              # 2

#             image = cv2.imread(os.path.join(images_path, img))

#             ss.setBaseImage(image)                                       # 3
#             ss.switchToSelectiveSearchFast(base_k=10)
#             ssresults = ss.process()
#             imout = image.copy()

#             # cv2.imshow("imout: ", imout)
#             # cv2.waitKey(0),
#             # cv2.destroyAllWindows()


#             # ele1n = os.path.join(images_path, img)

#             # In openCV, consider a square window facing us, 
#             # its left side will be y axis and top side will be x axis
#             # top left corner is considered as (0, 0)

#             iou_07 = 0
#             iou_03 = 0
#             for i in ssresults:
#                 x1, y1, w, h = i                                          # 4
#                 bb2 = {"x1":x1, "y1":y1, "x2":x1+w, "y2":y1+h}            
#                 img = img.split(".")[0]
#                 for j in img_bb[img]:                                     # For each true bb in img_bb, we calculate iou for the bb region proposed by ss
#                     x, y, xn, yn = j                                      # 5
#                     bb1 = {"x1":x, "y1":y, "x2":xn, "y2":yn}              # 6
#                     iou = gt(bb1, bb2)                                    # Calculating iou
#                     if iou > 0.7 and iou_07 < 30:                         # 7
#                         # print("iou07: bb: x1, y1, w, h, x1+w, y1+h:", x1, y1, w, h, x1+w, y1+h)
#                         timage = imout[y1:y1+h,x1:x1+w]
#                         resized = cv2.resize(timage, (224, 224), interpolation=cv2.INTER_AREA)
#                         x_new_images.append(resized)
#                         y_new_labels.append(1)
#                         iou_07 += 1
#                         # cv2.imshow("resized: ", resized)
#                         # cv2.waitKey(0)
#                         # cv2.destroyAllWindows()
#                         # cv2.rectangle(image, (bb1["x1"], bb1["y1"]), (bb1["x2"], bb1["y2"]), (255, 0, 0), 2)
#                         # cv2.rectangle(image, (bb2["x1"], bb2["y1"]), (bb2["x2"], bb2["y2"]), (0, 255, 0), 1)
#                     elif iou < 0.3 and iou_03 < 30:
#                         timage = imout[y1:y1+h, x1:x1+h]
#                         resized = cv2.resize(timage, (224, 224), interpolation=cv2.INTER_AREA)
#                         x_new_images.append(resized)
#                         y_new_labels.append(0)
#                         iou_03 += 1
#             # cv2.imshow(img, image)
#             # cv2.waitKey(0)
#             # cv2.destroyAllWindows()
#             # print("for image: ", img, " there are _ iou07 and _ iou03: ", iou_07, iou_03)
#             total_images_extracted += iou_03 + iou_07
#         no_images += 1
#     except Exception as e:
#         print(e)
#         print("Error in image: ", img)
#         continue